In [1]:
import pandas as pd

In [2]:
#loading dataset
data = pd.read_excel("/kaggle/input/tata-online-retail-dataset/Online Retail Data Set.xlsx")
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


**Exploratory Data Analysis**

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [4]:
data.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [5]:
data.drop_duplicates(inplace = True)

In [6]:
data.dropna(subset = ['CustomerID'], inplace = True)

In [7]:
#to check if there is any negative value in 'Quantity' column
(data['Quantity']<0).any()

True

In [8]:
# Filter out rows with negative values in the 'Quantity' column
data = data[data['Quantity'] >= 0]

In [9]:
# Calculate the total value of each transaction
data['Total'] = data['Quantity'] * data['UnitPrice']

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 392732 entries, 0 to 541908
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    392732 non-null  object        
 1   StockCode    392732 non-null  object        
 2   Description  392732 non-null  object        
 3   Quantity     392732 non-null  int64         
 4   InvoiceDate  392732 non-null  datetime64[ns]
 5   UnitPrice    392732 non-null  float64       
 6   CustomerID   392732 non-null  float64       
 7   Country      392732 non-null  object        
 8   Total        392732 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(4)
memory usage: 30.0+ MB


In [11]:
data["InvoiceDate"].max()

Timestamp('2011-12-09 12:50:00')

In [12]:
data["InvoiceDate"].min()

Timestamp('2010-12-01 08:26:00')

# Calculating RFM

In [13]:
import datetime as dt
recent= data["InvoiceDate"].max()
recent

Timestamp('2011-12-09 12:50:00')

In [14]:
# creating RFM table
rfmTable = data.groupby('CustomerID').agg({'InvoiceDate': lambda x: (recent - x.min()).days, # Recency
                                        'InvoiceNo': 'count',      # Frequency
                                        'Total': lambda x: x.sum()}).reset_index()
rfmTable

,CustomerID,InvoiceDate,InvoiceNo,Total
0,12346.0,325,1,77183.60
1,12347.0,366,182,4310.00
2,12348.0,357,31,1797.24
3,12349.0,18,73,1757.55
4,12350.0,309,17,334.40
...,...,...,...,...
4334,18280.0,277,10,180.60
4335,18281.0,180,7,80.82
4336,18282.0,125,12,178.05
4337,18283.0,336,721,2045.53


In [15]:
# renaming the columns
rfmTable.rename(columns={"InvoiceDate": "Recency",
                          "InvoiceNo":"Frequency",
                          "Total":"Monetary"},inplace=True)

rfmTable

,CustomerID,Recency,Frequency,Monetary
0,12346.0,325,1,77183.60
1,12347.0,366,182,4310.00
2,12348.0,357,31,1797.24
3,12349.0,18,73,1757.55
4,12350.0,309,17,334.40
...,...,...,...,...
4334,18280.0,277,10,180.60
4335,18281.0,180,7,80.82
4336,18282.0,125,12,178.05
4337,18283.0,336,721,2045.53


In [16]:
# highest frequency
rfmTable['Frequency'].max()

7676

In [17]:
# Filter the DataFrame based on the condition
rfmTable[rfmTable['Frequency'] == 7676]

,CustomerID,Recency,Frequency,Monetary
4011,17841.0,372,7676,40519.84


# RFM Scoring

In [18]:
# creating segments
recency=range(5,0,-1)
r_quartile=pd.qcut(rfmTable["Recency"],q=5,labels=recency).astype(int)
frequency=range(1,6)
f_quartile=pd.qcut(rfmTable["Frequency"].rank(method='first'),q=5,labels=frequency).astype(int)
monetary=range(1,6)
m_quartile=pd.qcut(rfmTable["Monetary"],q=5,labels=monetary).astype(int)


In [19]:
rfmTable["R"]=r_quartile
rfmTable["F"]=f_quartile
rfmTable["M"]=m_quartile
rfmTable[["R","F","M"]]
rfmTable["rfm_score"]=rfmTable["R"]+rfmTable['F']+rfmTable["M"]
rfmTable

,CustomerID,Recency,Frequency,Monetary,R,F,M,rfm_score
0,12346.0,325,1,77183.60,2,1,5,8
1,12347.0,366,182,4310.00,1,5,5,11
2,12348.0,357,31,1797.24,1,3,4,8
3,12349.0,18,73,1757.55,5,4,4,13
4,12350.0,309,17,334.40,2,2,2,6
...,...,...,...,...,...,...,...,...
4334,18280.0,277,10,180.60,3,1,1,5
4335,18281.0,180,7,80.82,4,1,1,6
4336,18282.0,125,12,178.05,4,1,1,6
4337,18283.0,336,721,2045.53,2,5,4,11


# Segmenting on the basis of RFM Score

In [20]:
def assign_label(data, r_rule, fm_rule, label, colname='rfm_segment'):
    data.loc[(data['R'].between(r_rule[0], r_rule[1]))
            & (data['F'].between(fm_rule[0], fm_rule[1])), colname]=label
    return data

In [21]:

rfmTable['RFM_Label'] = rfmTable.R.map(str)+" " \
                            + rfmTable.F.map(str)+" "  \
                            + rfmTable.M.map(str)+" " 

In [22]:
rfmTable = assign_label(rfmTable, (5,5), (4,5), 'Campions')
rfmTable = assign_label(rfmTable, (3,4), (4,5), 'Loyal customers')
rfmTable = assign_label(rfmTable, (4,5), (2,3), 'Potential loyalist')
rfmTable = assign_label(rfmTable, (5,5), (1,1), 'New customers')
rfmTable = assign_label(rfmTable, (4,4), (1,1), 'Promising')
rfmTable = assign_label(rfmTable, (3,3), (3,3), 'Needing attention')
rfmTable = assign_label(rfmTable, (3,3), (1,2), 'About to sleep')
rfmTable = assign_label(rfmTable, (1,2), (3,4), 'At risk')
rfmTable = assign_label(rfmTable, (1,2), (5,5), 'Can\'t loose them')
rfmTable = assign_label(rfmTable, (1,2), (1,2), 'Hibernating')
rfmTable


,CustomerID,Recency,Frequency,Monetary,R,F,M,rfm_score,RFM_Label,rfm_segment
0,12346.0,325,1,77183.60,2,1,5,8,2 1 5,Hibernating
1,12347.0,366,182,4310.00,1,5,5,11,1 5 5,Can't loose them
2,12348.0,357,31,1797.24,1,3,4,8,1 3 4,At risk
3,12349.0,18,73,1757.55,5,4,4,13,5 4 4,Campions
4,12350.0,309,17,334.40,2,2,2,6,2 2 2,Hibernating
...,...,...,...,...,...,...,...,...,...,...
4334,18280.0,277,10,180.60,3,1,1,5,3 1 1,About to sleep
4335,18281.0,180,7,80.82,4,1,1,6,4 1 1,Promising
4336,18282.0,125,12,178.05,4,1,1,6,4 1 1,Promising
4337,18283.0,336,721,2045.53,2,5,4,11,2 5 4,Can't loose them


**Champions** - bought recently, buy often and spend the most

**Loyal Customers** - spend good money and often, responsive to promotions

**Potential Loyalist** - recent customers, but spent a good amount and bought more than once

**New Customers** - bought most recently, but not often

**Promising** - recent shoppers, but haven’t spent much

**Needing Attention** - above average recency, frequency and monetary values; may not have bought very recently though

**About To Sleep** - below average recency, frequency and monetary values; will lose them if not reactivated

**At Risk** - spent big money and purchased often but long time ago; need to bring them back

**Can't Loose Them** - made biggest purchases, and often but haven’t returned for a long time

**Hibernating** - last purchase was long back, low spenders and low number of orders

# Visualizing

In [23]:
#Eliminating Future warning
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)


In [24]:
import plotly.graph_objects as go
import plotly.express as px

# Create an interactive histogram for recency distribution
fig = px.histogram(rfmTable, x='Recency', nbins=30, title='Recency Distribution')

# Update layout with title and customization
fig.update_layout(
    title_text='Recency Distribution',
    xaxis_title='Recency',
    yaxis_title='Count',
    bargap=0.1,  # Gap between bars
    barmode='overlay',  # Overlay bars
    template='plotly_dark'  # Dark theme
)

# Show the plot
fig.show()


In [25]:
# Create an interactive histogram for frequency distribution
fig_frequency = px.histogram(rfmTable, x='Frequency', nbins=30, title='Frequency Distribution')

# Update layout for frequency histogram
fig_frequency.update_layout(
    title_text='Frequency Distribution',
    xaxis_title='Frequency',
    yaxis_title='Count',
    bargap=0.1,
    barmode='overlay',
    template='plotly_dark'
)

# Show the frequency histogram
fig_frequency.show()

In [26]:
# Create an interactive histogram for monetary distribution
fig_monetary = px.histogram(rfmTable, x='Monetary', nbins=30, title='Monetary Distribution')

# Update layout for monetary histogram
fig_monetary.update_layout(
    title_text='Monetary Distribution',
    xaxis_title='Monetary',
    yaxis_title='Count',
    bargap=0.1,
    barmode='overlay',
    template='plotly_dark'
)

# Show the monetary histogram
fig_monetary.show()

In [27]:
import plotly.graph_objects as go

# Create a 3D scatter plot
fig = go.Figure(data=[go.Scatter3d(
    x=rfmTable['Recency'],
    y=rfmTable['Frequency'],
    z=rfmTable['Monetary'],
    mode='markers',
    marker=dict(
        size=8,
        color=rfmTable['rfm_score'],  # Color by segment 
        colorscale='Viridis',  # Choose a color scale 
        opacity=0.8
    ),
    text=rfmTable['CustomerID'],  # Display CustomerID on hover
)])

# Update layout with axis labels and title
fig.update_layout(
    scene=dict(
        xaxis_title='Recency',
        yaxis_title='Frequency',
        zaxis_title='Monetary'
    ),
    title='RFM Score Visualization'
)

# Show the plot
fig.show()


In [ ]:
import plotly.express as px

# Create a 3D scatter plot
fig = px.scatter_3d(rfmTable, x='Recency', y='Frequency', z='Monetary', color='rfm_segment',
                    symbol='rfm_segment', opacity=0.7, size_max=10,
                    title='Customer Segmentation based on RFM Scores')

# Update layout with axis labels
fig.update_layout(scene=dict(xaxis_title='Recency', yaxis_title='Frequency', zaxis_title='Monetary'))

# Show the plot
fig.show()


In [31]:
import plotly.express as px

# Create an interactive scatter plot for RFM scores with segment labels
fig = px.scatter(rfmTable, x='Recency', y='Frequency', color='rfm_segment', hover_data=['CustomerID'],
                 title='Customer Segmentation based on RFM Scores')

# Update layout
fig.update_layout(xaxis_title='Recency', yaxis_title='Frequency')

# Show the plot
fig.show()


In [32]:
import plotly.express as px

# Calculate the count of each segment
segment_counts = rfmTable['rfm_segment'].value_counts()

# Create an interactive pie chart for segment distribution
fig = px.pie(names=segment_counts.index, values=segment_counts.values,
             title='Segment Distribution')

# Show the plot
fig.show()
